In [1]:
from itertools import combinations
import pandas as pd
import lxml.etree
import networkx as nx
import numpy as np
np.random.seed(18012023)
import scipy.spatial.distance as scidist
import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_excel('../data/metadata_corrected.xlsx')
df = df[df['exclude'] != 'x']
df.sample(10)

,id,title,author,date,provenance,date_range,genre,subgenre,exclude
265,van_sente_brandane,Van sente Brandane,NaN,1402.5,cdrom-mnl,1380-1425,Epiek,Heiligenleven,NaN
63,florigout_fragm_l,Florigout,NaN,1387.5,cdrom-mnl,1375-1400,Epiek,Ridder,NaN
111,loyhier_en_malaert_fragm_a,Loyhier en Malaert,NaN,1375.0,cdrom-mnl,1350-1400,Epiek,Karel,NaN
86,historie_van_gaver_capeel,Historie van Gaver Capeel,NaN,1510.0,cdrom-mnl,1500-1520,Epiek,Ridder,NaN
167,Reinout van Montalbaen,Reinout van Montalbaen,NaN,1288.0,CG1,1300-1276,Epiek,Karel,NaN
210,spiegel_der_jongers,Spiegel der jongers,Lambertus Goetman,1515.0,cdrom-mnl,1510-1520,Epiek,Didactiek,NaN
209,spel_van_de_antichrist,Spel van de Antichrist,NaN,1425.0,cdrom-mnl,1400-1450,Dramatiek,NaN,NaN
123,madelgijs_fragm_p,Madelgijs,NaN,1387.5,cdrom-mnl,1375-1400,Epiek,Karel,NaN
165,queeste_van_den_grale,Queeste van den Grale,NaN,1325.0,cdrom-mnl,1300-1350,Epiek,Arthur,NaN
238,strofische_gedichten,Strofische gedichten,Hadewijch,1350.0,cdrom-mnl,1340-1360,Lyriek,NaN,NaN


In [3]:
works = {}

for title, group in df.groupby('title'):
    words = []
    for id_ in sorted(group['id']):
        try:
            tree = lxml.etree.parse(f'../data/xml/{id_}.xml')
        except OSError:
            print(f'- Could not load {id_}')
            continue
        
        for line_node in tree.iterfind('//l'):
            tokens = line_node.attrib['tokens']
            lemmas = [l.text for l in line_node.iterfind('.//lemma')]
            pos_tags = [p.text for p in line_node.iterfind('.//pos')]
            
            for lemma, pos_tag in zip(lemmas, pos_tags):
                for l, p in zip(lemma.split('+'), pos_tag.split('+')):
                        if p != 'n(prop)':
                            words.append(l)
    
    works[title] = words

In [4]:
length = 3000

titles, texts = [], []

for title, words in works.items():
    si, ei = 0, length
    while ei < len(words):
        titles.append(title)
        texts.append(' '.join(words[si:ei]))
        si += length
        ei += length
        
df = pd.DataFrame(zip(titles, texts), columns=('title', 'text'))
df.sample(3)

,title,text
1019,Roman der Lorreinen,en haar ingang zijn zij benemen van spijs en v...
276,Der naturen bloeme,zij te prijg zingen eer zij zij verwinnen gaan...
1363,Spiegel historiael (P1-P4 [Maerlant]),van de paus legaat met hij in wie bisdom dat s...


In [5]:
vec = TfidfVectorizer(min_df=2, token_pattern=r"(?u)\b\w+\b")
X = vec.fit_transform(df['text']).toarray()
X.shape

(1624, 16233)

In [6]:
titles = sorted(df['title'].unique())
distances = np.zeros((len(titles), len(titles)))
for t1, t2 in tqdm.tqdm(list(combinations(titles, 2))):
    A = X[df['title'] == t1]
    B = X[df['title'] == t2]
    distance = scidist.cdist(A, B, metric='cosine').mean()
    distances[titles.index(t1), titles.index(t2)] = distance
    distances[titles.index(t2), titles.index(t1)] = distance

100%|██████████| 6555/6555 [00:35<00:00, 182.52it/s]


In [7]:
G = nx.Graph() 
for t1, v in zip(titles, distances):
    tops = v.argsort()[1:4] # ignore self-distance; take top-3 (like Eder)
    for t2 in tops:
        t2 = titles[t2]
        if G.has_edge(t1, t2):
            G[t1][t2]['weight'] += 1
        else:
            G.add_edge(t1, t2, weight=1)
print(f"N nodes = {G.number_of_nodes()}, N edges = {G.number_of_edges()}")
nx.write_gexf(G, "../network.gexf")

N nodes = 115, N edges = 309
